In [1]:
import pandas as pd
import random
import numpy as np


valid_patterns = {
'A': [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
'B': [1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0],
'C': [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1],
'D': [1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0],
'E': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1],
'F': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
'G': [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0],
'H': [1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
'I': [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1],
'J': [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0],
'K': [1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1],
'L': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1],
'M': [1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
'N': [1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1],
'O': [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0],
'P': [1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
'Q': [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1],
'R': [1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1],
'S': [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
'T': [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
'U': [1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0],
'V': [1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
'W': [1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1],
'X': [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
'Y': [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
'Z': [1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1]

}


# Function to create variations by making small changes
def create_variations(pattern, num_variations=5):
    variations = [pattern]
    
    for _ in range(num_variations - 1):  # We already have the original pattern
        new_pattern = pattern.copy()
        change_type = random.choice(['flip', 'shift', 'rotate'])
        
        if change_type == 'flip':
            # Flip a random pixel (change 0 to 1 or 1 to 0)
            idx = random.randint(0, 24)
            new_pattern[idx] = 1 - new_pattern[idx]
        
        elif change_type == 'shift':
            # Shift a random row or column
            shift_type = random.choice(['row', 'col'])
            if shift_type == 'row':
                row = random.randint(0, 4)
                shift_direction = random.choice([-1, 1])  # shift left or right
                new_pattern[row*5:(row+1)*5] = np.roll(new_pattern[row*5:(row+1)*5], shift_direction)
            else:
                col = random.randint(0, 4)
                col_values = [new_pattern[i*5 + col] for i in range(5)]
                shift_direction = random.choice([-1, 1])  # shift up or down
                col_values = np.roll(col_values, shift_direction)
                for i in range(5):
                    new_pattern[i*5 + col] = col_values[i]
        
        elif change_type == 'rotate':
            # Rotate the grid slightly (90 degrees)
            new_pattern = np.array(new_pattern).reshape(5, 5)
            new_pattern = np.rot90(new_pattern)  # Rotate 90 degrees
            new_pattern = new_pattern.flatten()
        
        variations.append(new_pattern)
    
    return variations

# Create variations for each valid letter pattern
patterns = []
for letter, pattern in valid_patterns.items():
    variations = create_variations(pattern, num_variations=5)  # Create 5 variations for each letter
    patterns.extend(variations)

# Define labels for the letters A-Z (mapped to 1 for valid patterns)
valid_labels = [1] * len(patterns)

# Generate 100 unique invalid patterns (patterns that do not correspond to any valid letter)
valid_patterns_set = {tuple(pattern) for pattern in valid_patterns.values()}
invalid_patterns = []
while len(invalid_patterns) < 100:
    pattern = tuple(random.choices([0, 1], k=25))  # 5x5 grid flattened to 25 elements
    if pattern not in valid_patterns_set:
        invalid_patterns.append(list(pattern))

# Invalid labels (0 for invalid patterns)
invalid_labels = [0] * len(invalid_patterns)

# Combine valid patterns (with variations) and invalid patterns with labels
all_patterns = patterns + invalid_patterns
all_labels = valid_labels + invalid_labels

# Create a DataFrame
df = pd.DataFrame(all_patterns, columns=[f'Pixel_{i+1}' for i in range(25)])  # 5x5 grid flattened to 25 pixels
df['label'] = all_labels

# Shuffle the dataset to avoid bias in ordering
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the dataset to a CSV file
df.to_csv("balanced_alphabet_dataset_with_variations.csv", index=False)
print("Balanced and shuffled dataset with variations saved as 'balanced_alphabet_dataset_with_variations.csv'")


Balanced and shuffled dataset with variations saved as 'balanced_alphabet_dataset_with_variations.csv'
